In [ ]:
!pip install pyresparser fitz PyMuPDF pypdf2 >/dev/null

In [ ]:
!ln -s "NOUVELLE FLORE DU LIBAN ET DE LA SYRIE.pdf" document.pdf

In [ ]:
import fitz
document = 'document.pdf'
filename = document
search_term = "the"
pdf_document = fitz.open(filename)

In [ ]:
doc = pdf_document
dir(doc)

In [ ]:
doc.getToC()

In [ ]:
doc.searchPageFor(528, 'Pteranthus')

In [ ]:
# https://pymupdf.readthedocs.io/en/latest/faq/#how-to-analyze-font-characteristics

import fitz
import pprint, re

def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)


doc = fitz.open("document.pdf")
page = doc[528]

In [ ]:
#header = 'Family	Genus	epithet	author	syn.	Plate	description	Floraison from	Floraison to	Floraison notes	L.	S.	Aire géogr.	Notes'
header = 'Family	Genus	epithet	author	syn.	Plate	description	Floraison	L.	S.	Aire géogr.	Notes'
columns = header.split('	')
columns

In [ ]:
class Face:
    def __init__(self, font, size, color, flags=0):
        self.font = font
        self.size = size
        self.color = color
        self.flags = flags
        self.style = flags_decomposer(self.flags)
    def __str__(self):
        return f"Font: '{self.font}' ({self.style}), size {self.size:g}, color " + ("#%06x" % self.color)
print(Face("Times",10.929, 0, 4))

def isfamily(t, fp):
    font = "Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000"
    if re.match('^[A-Z]+$', t) or str(font) == fp:
        #print(f'''Text: {t}\n{fp}\n\n''')
        return True
data = {}
item = dict.fromkeys(columns)
print(item)
p = (doc[x] for x in range(516,604))
p = (doc[x] for x in range(527,604))
p

In [ ]:
import re

def floraison(doc, pagenum=0, block_start=0):
    for page in next(p):
        blocks = page.getText("dict", flags=11)["blocks"]
        for x,b in enumerate(blocks[block_start:]):  # iterate through the text blocks
            for y,l in enumerate(b["lines"]):  # iterate through the text lines
                for z,s in enumerate(l["spans"]):  # iterate through the text spans
                    s["style"] = flags_decomposer(s["flags"])  # readable font flags
                    face = Face(s["font"], s["size"], s["color"], s["flags"])
                    if re.match("Floraison", s["text"]):
                        yield (page.number, x, y, z, s["text"])

p = (doc[x] for x in range(516,540))
next(floraison(doc, x))

In [ ]:
p = (doc[x] for x in range(527,604))
p

In [ ]:
page.number

In [ ]:
page = next(p)
# read page text as a dictionary, suppressing extra spaces in CJK fonts
blocks = page.getText("dict", flags=11)["blocks"]
for x,b in enumerate(blocks):  # iterate through the text blocks
    for y,l in enumerate(b["lines"]):  # iterate through the text lines
        for z,s in enumerate(l["spans"]):  # iterate through the text spans
            s["style"] = flags_decomposer(s["flags"])  # readable font flags
            face = Face(s["font"], s["size"], s["color"], s["flags"])
            if isfamily(s["text"], face) and len(l["spans"]) == 1:
                item['Family'] = s["text"]
                print(x, len(blocks), y, z, s["text"], face)
                break
            if not item['Family']:
                break
            if s['font'] == 'Times-Bold':
                item['Genus'], *item['epithet'] = s["text"].split(' ')
                item['epithet'] = item['epithet'][0] if item['epithet'] else l["spans"][1]["text"]
                currsize = s['size']
#                print(x, len(blocks), y, z, s["text"], font_properties)
                print(x, len(blocks), y, len(b["lines"]), z, s["text"], face)
                nn, xx, yy, zz, tt = next(floraison(page, x))
                item['Floraison'] = tt
#                 for q in range(x, max(x+15, len(blocks))):
#                     for ll in blocks[q]["lines"]:
#                         s = ll["spans"][0]
#                         text, *rest = s['text'].split(' ')
#                         if text == "Floraison:":
#                             face = Face(s["font"], s["size"], s["color"], s["flags"])
#                             print(face, s['text'])
#                             for ss in ll["spans"]:
#                                 #print(ss['text'], end=" ")
#                                 pass
                from pprint import pprint
                print(x, len(blocks), y, z, pprint(item))
                #print([a["text"] for a in l["spans"]])
                print(); print(); 
                break


In [ ]:
search = ((x, doc.searchPageFor(x, "Floraison", 100, False)) for x in range(516,520))

In [ ]:
next(search)

In [ ]:
p = (doc[x] for x in range(516,520))

In [ ]:
page = next(p)
rects = page.searchFor("Floraison")
rects[0]

In [ ]:
page.getTextBlocks()

In [ ]:
d = iter(doc)

In [ ]:
next(d).getTextBlocks()

In [ ]:
doc[621].getTextBlocks()

In [ ]:
doc[621].getText()

In [ ]:
page.getFontList()